# Install and import libraries

In [13]:
%pip install pydot
%pip install tensorflow
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from pathlib import Path

from constants import (
    DATA_INPUT_PATH,
    MODEL_PATH,
    METADATA_PATH,
)

CLASSES = {
    0: "thumbs-up",
    1: "peace-sign",
    2: "gun-fingers",
    3: "fist",
    4: "open-palm"
}

# Read the files in the data dir

In [27]:
# Read all of the files in the data folder
files_in_folder = Path(DATA_INPUT_PATH).glob("*.csv")

files = [x for x in files_in_folder]
print([file for file in files])

[WindowsPath('data/emg_gestures_data_20241209_150423.csv')]


## Convert .csv(s) to dataframes and concatenate

In [28]:
# Read the data from the files
dfs = []

for file in files:
    df = pd.read_csv(str(file))
    dfs.append(df)
        
# Convert the data to a DataFrame
df = pd.concat([x for x in dfs], axis=0)

columns = ["gesture_id", "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8"]
df = df[columns]

print(df.head())

# Before removing duplicates
print(f"Shape of dataframe before removing duplicates {df.shape}")
# Remove duplicates
df = df.drop_duplicates()
print(f"Shape of dataframe after removing duplicates {df.shape}")

   gesture_id   s1   s2   s3   s4   s5   s6   s7   s8
0           0  440  438  582  388  266  128  244  506
1           0  290  266  285  172  125   64  114  333
2           0  245  231  208  130   95   46   97  205
3           0  244  210  129   93   73   40   82  180
4           0  248  184  104   73   60   36   84  171
Shape of dataframe before removing duplicates (1227, 9)
Shape of dataframe after removing duplicates (1227, 9)


In [29]:
# Parameters
window_size = 10  # Number of past samples to include

# Extract feature columns (excluding label, if it exists)
feature_columns = [col for col in df.columns if col != 'Label']
features = df[feature_columns].to_numpy()

# Convert features into a 3D array: [samples, features, time]
n_samples = len(features)
n_features = len(feature_columns)
reshaped_data = []

for i in range(n_samples):
    # Get the last `window_size` rows or fewer for each sample
    window = features[max(i - window_size + 1, 0):i + 1]
    
    # Pad the window if it has fewer than `window_size` rows
    if len(window) < window_size:
        padding = np.zeros((window_size - len(window), n_features))
        window = np.vstack((padding, window))
    
    reshaped_data.append(window)

# Convert list to 3D numpy array
reshaped_data = np.stack(reshaped_data, axis=0)
reshaped_data = reshaped_data.transpose(0, 2, 1)
df[feature_columns] = np.sqrt(np.mean(np.square(reshaped_data), axis=2))
    
print(df.shape)
print(df.head())

(1227, 9)
   gesture_id          s1          s2          s3          s4          s5  \
0         0.0  139.140217  138.507762  184.044560  122.696373   84.116586   
1         0.0  166.643332  162.049375  204.926572  134.211773   92.941379   
2         0.0  183.772958  177.752918  215.223837  140.366663   97.675995   
3         0.0  199.314074  189.752734  219.055701  143.414434  100.366827   
4         0.0  214.188002  198.473424  221.510722  145.260456  102.144505   

          s6         s7          s8  
0  40.477154  77.159575  160.011250  
1  45.254834  85.165721  191.552865  
2  47.535250  90.521268  202.225122  
3  49.189430  94.162094  210.083317  
4  50.489603  97.837110  216.931095  


## Scale and clean the data, then train the model

In [30]:
X = df.drop(columns=['gesture_id'])
y = df['gesture_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the data for Conv1D
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# # filter for valid classes because the data is not clean
# valid_classes = [0, 2, 3]
# mask_train = y_train.isin(valid_classes)
# mask_test = y_test.isin(valid_classes)

# # apply the mask to the training and testing data
# X_train_filtered = X_train_scaled[mask_train]
# y_train_filtered = y_train[mask_train]

# X_test_filtered = X_test_scaled[mask_test]
# y_test_filtered = y_test[mask_test]

# one hot encode the target data
y_train_categorical = to_categorical(y_train, num_classes=8)
y_test_categorical = to_categorical(y_test, num_classes=8)

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(8, activation='softmax')
])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Compile the model with categorical crossentropy for multi-class classification
model.compile(
    optimizer=Adam(learning_rate=0.001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'],
)

# Train the model using the filtered training data
model.fit(
    X_train_scaled, 
    y_train_categorical, 
    epochs=100, 
    batch_size=64, 
    validation_split=0.2, 
    callbacks=[early_stopping]
)

# Evaluate the model on the filtered test set
test_loss, test_acc = model.evaluate(X_test_scaled, y_test_categorical)

print(f"Test accuracy: {test_acc}")

Epoch 1/100


c:\Users\willi\Desktop\Uni\3rd_year\NLP\CW1\.conda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2250 - loss: 2.1390 - val_accuracy: 0.7563 - val_loss: 1.7650
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8104 - loss: 0.7337 - val_accuracy: 0.8376 - val_loss: 1.4454
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9205 - loss: 0.3948 - val_accuracy: 0.8579 - val_loss: 1.2139
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9421 - loss: 0.2949 - val_accuracy: 0.8832 - val_loss: 1.0481
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9544 - loss: 0.2173 - val_accuracy: 0.9289 - val_loss: 0.9163
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9421 - loss: 0.2150 - val_accuracy: 0.9645 - val_loss: 0.8009
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9404 - loss: 0.1761 - val_accuracy: 0.9289 - val_loss: 0.7123
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9402 - loss: 0.1868 - val_accuracy: 0.9797 - val_loss: 0.

In [33]:
# Save the model
model.save(f"model/{MODEL_PATH}")

# Save the scaler and the column names to a pickle file
with open(f"model/{METADATA_PATH}", 'wb') as f:
    pickle.dump((scaler, X_train.columns), f)

In [34]:
# thumbs-up then peace-sign
emg_data = [
    [160,245,126,32,28,25,26,99], # thumbs-up
    [67,197,559,104,41,82,257,169], # peace-sign
    [205,440,165,40,27,83,229,226], # gun-fingers
    [416,434,134,71,36,48,102,103] # fist
]

for data in emg_data:
    emg_features_df = pd.DataFrame([data], columns=X_train.columns)

    emg_features_scaled = scaler.transform(emg_features_df)
    emg_features_reshaped = emg_features_scaled.reshape(1, -1)

    prediction = model.predict(emg_features_reshaped)
    predicted_class = np.argmax(prediction)

    print(f"Predicted class: {predicted_class} - {CLASSES[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted class: 2 - gun-fingers
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted class: 1 - peace-sign
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted class: 1 - peace-sign
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted class: 3 - fist
